# Blockchain Data Discovery: Visualization

This notebook visualizes the insights from the Arbitrum blockchain data (Uniswap V3 Swaps & Aave V3 Liquidations).

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

DB_PATH = "blockchain.db"

# Constants
DECIMALS = {'WETH': 18, 'USDC': 6, 'AAVE': 18, 'DAI': 18, 'WBTC': 8}
PRICES = {'WETH': 3100.0, 'USDC': 1.0, 'AAVE': 90.0, 'DAI': 1.0, 'WBTC': 95000.0}

def get_connection():
    return sqlite3.connect(DB_PATH)

def normalize_amount(amount, token_symbol):
    decimals = DECIMALS.get(token_symbol, 18)
    return float(amount) / (10 ** decimals)

## 1. MEV Sandwich Attack Analysis

In [ ]:
conn = get_connection()
query = """
SELECT transaction_hash, block_number, block_timestamp, log_index, sender, recipient, amount0, amount1, sqrt_price_x96, liquidity
FROM uniswap_swaps_v2
ORDER BY block_number, log_index
"""
df_swaps = pd.read_sql_query(query, conn)
df_swaps['amount0'] = df_swaps['amount0'].apply(float)
df_swaps['amount1'] = df_swaps['amount1'].apply(float)
df_swaps['sqrt_price_x96'] = df_swaps['sqrt_price_x96'].apply(float)
df_swaps['timestamp'] = pd.to_datetime(df_swaps['block_timestamp'], unit='s')

# Pool Classification & Volume Calculation
def classify_pool(row):
    p = row['sqrt_price_x96']
    if p < 1e27: return 'WETH/USDT'
    elif p < 1e31: return 'WBTC/WETH'
    else: return 'USDC/WETH'

df_swaps['pool'] = df_swaps.apply(classify_pool, axis=1)

def calc_volume(row):
    pool = row['pool']
    a0 = abs(row['amount0'])
    a1 = abs(row['amount1'])
    if pool == 'WETH/USDT': # T0=WETH(18), T1=USDT(6)
        return max((a0/1e18)*PRICES['WETH'], (a1/1e6)*1.0)
    elif pool == 'WBTC/WETH': # T0=WBTC(8), T1=WETH(18)
        return max((a0/1e8)*PRICES['WBTC'], (a1/1e18)*PRICES['WETH'])
    elif pool == 'USDC/WETH': # T0=USDC(6), T1=WETH(18)
        return max((a0/1e6)*PRICES['USDC'], (a1/1e18)*PRICES['WETH'])
    return 0.0

df_swaps['volume_usd'] = df_swaps.apply(calc_volume, axis=1)

# Detect Sandwiches (Updated with Profit Check)
grouped = df_swaps.groupby('block_number')
sandwiches = []

for block_num, group in grouped:
    if len(group) < 3: continue
    group = group.sort_values('log_index')
    txs = group.to_dict('records')
    
    for i in range(len(txs) - 2):
        tx1 = txs[i]
        dir1 = 1 if tx1['amount0'] < 0 else -1
        for j in range(i + 1, len(txs) - 1):
            tx2 = txs[j]
            dir2 = 1 if tx2['amount0'] < 0 else -1
            if dir1 != dir2: continue
            for k in range(j + 1, len(txs)):
                tx3 = txs[k]
                dir3 = 1 if tx3['amount0'] < 0 else -1
                if dir1 == dir3: continue
                
                if tx1['recipient'] == tx3['recipient']:
                    # Calculate Profit
                    profit0_raw = tx1['amount0'] + tx3['amount0']
                    profit1_raw = tx1['amount1'] + tx3['amount1']
                    # Normalize (Corrected Mapping: Token0=WETH, Token1=USDC)
                    profit0 = normalize_amount(profit0_raw, 'WETH')
                    profit1 = normalize_amount(profit1_raw, 'USDC')
                    profit_usd = (profit0 * PRICES['WETH']) + (profit1 * PRICES['USDC'])
                    
                    if True: # Show all potential sandwiches
                        sandwiches.append({
                            'block_number': block_num,
                            'attacker': tx1['recipient'],
                            'profit_usd': profit_usd,
                            'victim_tx_hash': tx2['transaction_hash']
                        })

df_mev = pd.DataFrame(sandwiches)
print(f"Found {len(df_mev)} profitable sandwiches")
if not df_mev.empty:
    display(df_mev.head())

In [ ]:
if not df_mev.empty:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df_mev, x='profit_usd', bins=10, kde=True, color='orange')
    plt.title('Distribution of MEV Sandwich Profits (USD)')
    plt.xlabel('Profit (USD)')
    plt.ylabel('Frequency')
    plt.show()

## 2. Volume & Activity Analysis

In [ ]:
# 1. Volume Distribution (Log Scale)
plt.figure(figsize=(10, 6))
sns.histplot(data=df_swaps, x='volume_usd', bins=50, log_scale=True, color='teal')
plt.title('Distribution of Swap Sizes (USD) - Log Scale')
plt.xlabel('Swap Volume (USD)')
plt.ylabel('Count')
plt.show()

# 2. Hourly Activity
df_hourly = df_swaps.set_index('timestamp').resample('H').agg({'volume_usd': 'sum', 'transaction_hash': 'count'})

fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Time')
ax1.set_ylabel('Total Volume (USD)', color=color)
ax1.plot(df_hourly.index, df_hourly['volume_usd'], color=color, marker='o')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Transaction Count', color=color)
ax2.plot(df_hourly.index, df_hourly['transaction_hash'], color=color, linestyle='--', alpha=0.7)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Hourly Trading Volume & Activity')
fig.tight_layout()
plt.show()

## 3. Liquidation Analysis

In [ ]:
query_liq = """
SELECT transaction_hash, collateral_asset, debt_asset, user, debt_covered, collateral_amount, liquidator
FROM aave_liquidations_v2
"""
df_liq = pd.read_sql_query(query_liq, conn)
conn.close()

df_liq['debt_covered'] = df_liq['debt_covered'].apply(float)
df_liq['debt_norm'] = df_liq['debt_covered'] / 1e18 # Normalize
df_liq['est_profit_usd'] = df_liq['debt_norm'] * 0.05 * 3000 # Est 5% bonus

# Top Liquidators
top_liquidators = df_liq.groupby('liquidator')['est_profit_usd'].sum().sort_values(ascending=False).head(5)
top_liquidators = top_liquidators.reset_index()
top_liquidators['short_addr'] = top_liquidators['liquidator'].apply(lambda x: x[:6] + '...' + x[-4:])

plt.figure(figsize=(12, 6))
sns.barplot(data=top_liquidators, x='short_addr', y='est_profit_usd', palette='viridis')
plt.title('Top 5 Liquidators by Estimated Profit')
plt.xlabel('Liquidator Address')
plt.ylabel('Estimated Profit (USD)')
plt.xticks(rotation=45)
plt.show()